In [1]:
import os
!export LC_ALL=en_US.UTF-8
#os.environ['CUDA_VISIBLE_DEVICES'] = '3'
from embedding import *

In [2]:
import json
from pprint import pprint
import re
import numpy as np
import sys
import math
import random

import torch
try:
    import nltk
except:
    !pip install nltk
    import nltk
    nltk.download('punkt')
    nltk.download('stopwords')
np.random.seed(0)

In [3]:
data = json.load(open('../train-v1.1.json'))

In [4]:
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltkStopWords = stopwords.words('english')
punctuations = [',', '?', '.', '-',]

[nltk_data] Downloading package stopwords to /home/ra2630/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ra2630/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def extractor(data):
    contexts = []
    qas = []
    for i in range(len(data["data"])):
        for j in range(len(data["data"][i]["paragraphs"])):
            contexts.append(data["data"][i]["paragraphs"][j]["context"])
            qas.append(data["data"][i]["paragraphs"][j]["qas"])
    return (contexts,qas)

In [6]:
CapPassage = True

from nltk.tokenize import word_tokenize
contexts,qas = extractor(data)

def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll
    return (-1,-1)

def capPassage(passage,left, right,cap_length = 20):
    y = np.zeros(cap_length)
    left = max(0,left - int((cap_length - len(answer))/2))
    right = min(right + int((cap_length + len(answer))/2), cap_length)
    if(left < 0):
        left = 0
    if(right > len(passage)):
        right = len(passage)
    return passage[left:right]
    
def findAnsVec(answer,passage):
    ans = np.zeros((len(passage)))
    start,end = find_sub_list(answer,passage)
    if(start==-1):
        start = passage.index(answer[0])
        end = start + len(answer)
    ans[start:end] = 1
    return ans

In [7]:
n_words = 10
X_train_comp_all = []
X_train_comp_with_answer_marked_all = []
X_train_ans_all = []
X_train_comp_answer_label_all = []
X_train_sentence_label_all = []
Y_train_ques_all = []
invalid = 0
for i,context in enumerate(contexts):
    passage = word_tokenize(context.lower())

    for j,_ in enumerate(qas[i]):
        try:
            question = word_tokenize(qas[i][j]['question'].lower())
            answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
            start,end = find_sub_list(answer,passage)
            if start == -1:
                invalid = invalid+1
                continue
            marked_comp = np.zeros(len(passage))
            marked_comp[start:end] = 1
            left = max(0,start - n_words)
            right = min(len(passage), end + n_words)
            
            cappedPassage = passage[left:right]
            marked_comp = marked_comp[left:right]
            
            
            X_train_comp_all.append(cappedPassage)
            X_train_comp_with_answer_marked_all.append(marked_comp)
            X_train_ans_all.append(answer)
            Y_train_ques_all.append(question)
        except Exception as e:
            invalid = invalid+1
print(invalid)

2429


In [8]:
c = list(zip(X_train_comp_all,X_train_ans_all, Y_train_ques_all,X_train_comp_with_answer_marked_all))
np.random.shuffle(c)
X_train_comp_all_shuffled,X_train_ans_all_shuffled, Y_train_ques_all_shuffled, X_train_comp_with_answer_marked_all_shuffled = zip(*c)


In [9]:
X_train_comp_with_answer_marked_all_shuffled[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])

In [10]:
from collections import Counter
import operator
def findKMostFrequentWords(k):
    ctr = Counter([item for sublist in X_train_comp_all_shuffled for item in sublist] + [item for sublist in Y_train_ques_all_shuffled for item in sublist])
    sorted_ctr = sorted(ctr.items(), key=operator.itemgetter(1), reverse=True)
    return [item[0] for item in sorted_ctr[0:k]]

In [11]:
wordToTake = 200000
words = findKMostFrequentWords(100000)

In [12]:
_word_to_idx_reduced = {}
_idx_to_word_reduced = []


def _add_word_reduced(word):
    idx = len(_idx_to_word_reduced)
    _word_to_idx_reduced[word] = idx
    _idx_to_word_reduced.append(word)
    return idx



PAD_TOKEN = _add_word_reduced(PAD_WORD)
START_TOKEN = _add_word_reduced(START_WORD)
END_TOKEN = _add_word_reduced(END_WORD)
UNKNOWN_TOKEN = _add_word_reduced(UNKNOWN_WORD)




dimensions = glove.shape[1]
reduced_glove = []
reduced_glove.append(np.zeros(dimensions))
reduced_glove.append(-np.ones(dimensions))
reduced_glove.append(np.ones(dimensions))

for word in words:
    l = look_up_word(word)
    if(l != UNKNOWN_TOKEN):
        idx = _add_word_reduced(word)
        reduced_glove.append(glove[l])
        if(len(reduced_glove) == wordToTake):
            break
        
def look_up_word_reduced(word):
    return _word_to_idx_reduced.get(word, UNKNOWN_TOKEN)


def look_up_token_reduced(token):
    return _idx_to_word_reduced[token]

reduced_glove = np.array(reduced_glove)


In [13]:
common_words_set_tokens = set([look_up_word_reduced(i) for i in set(stopwords.words('english'))]) - {UNKNOWN_TOKEN, START_TOKEN}

In [14]:
len(common_words_set_tokens)

137

In [15]:
!hostname
print("Invalid = ",invalid)
print("Glove shape = ", reduced_glove.shape)

cuda2.cims.nyu.edu
Invalid =  2429
Glove shape =  (78658, 300)


In [16]:
examples_to_take_train = 32

X_train_comp = X_train_comp_all_shuffled[0:examples_to_take_train]
X_train_ans = X_train_ans_all_shuffled[0:examples_to_take_train]
Y_train_ques = Y_train_ques_all_shuffled[0:examples_to_take_train]
X_train_comp_with_answer_marked = X_train_comp_with_answer_marked_all_shuffled[0:examples_to_take_train]

In [17]:
max_document_len = len(max(X_train_comp,key=len))
max_answer_len = len(max(X_train_ans,key=len))
max_question_len = len(max(Y_train_ques,key=len)) + 1

In [18]:
document_tokens = np.full((examples_to_take_train, max_document_len), PAD_TOKEN,dtype=np.int32)
document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)

answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)

question_input_tokens = np.full((examples_to_take_train, max_question_len), PAD_TOKEN, dtype=np.int32)
question_output_tokens = np.full((examples_to_take_train, max_question_len), PAD_TOKEN, dtype=np.int32)
question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)


In [19]:
for i in range(examples_to_take_train):
    answer_labels[i, 0:len(X_train_comp_with_answer_marked[i])] = X_train_comp_with_answer_marked[i]
    for j, word in enumerate(X_train_comp[i]):
        document_tokens[i, j] = look_up_word_reduced(word)
    document_lengths[i] = len(X_train_comp[i])

    answer_lengths[i] = len(X_train_ans[i])
    
    question_input_words = ([START_WORD] + Y_train_ques[i])
    question_output_words = (Y_train_ques[i] + [END_WORD])

    for j, word in enumerate(question_input_words):
            question_input_tokens[i, j] = look_up_word_reduced(word)
    for j, word in enumerate(question_output_words):
        question_output_tokens[i, j] = look_up_word_reduced(word)
    question_lengths[i] = len(question_input_words)    

In [55]:
def context_to_indices_glove(X,max_len):
    
    m = len(X)                                 
    
    X_indices = np.full([m,max_len],look_up_word_reduced(PAD_WORD))
    
    for i in range(m):
        j = 0
        for w in X[i]:
            if(j>=max_len):
                break;
            
            X_indices[i, j] = look_up_word_reduced(w)
            j = j+1
        if(j < max_len):
            X_indices[i, j] = look_up_word_reduced(END_WORD)
    return X_indices

In [56]:
def generate_valid_tokens_vocab(X, Y):
    
    m = len(X)                                 
    
    X_indices = np.full([m,reduced_glove.shape[0]],PAD_TOKEN)
    lengths = np.zeros(m)
    
    for i in range(m):
        #s = set(X[i]).union(common_words_set_tokens) - {UNKNOWN_TOKEN}
        s = set(X[i]).union(set(Y[i])) - {UNKNOWN_TOKEN}
        for w in s:
            X_indices[i, w] = 1
        lengths[i] = len(s)
    return X_indices, lengths

In [57]:
document_tokens = context_to_indices_glove(X_train_comp, max_document_len)

In [58]:
valid_tokens, valid_tokens_length = generate_valid_tokens_vocab(document_tokens, question_output_tokens)

In [59]:
valid_tokens_length[1]

26.0

## Batch Data Preparation

In [60]:
max_question_len

20

In [324]:
import math

def createBatch(inputs,batch_size,shuffle=False):
    outputs = []
    start = 0
    while start < len(inputs[0]):
        end = min(len(inputs[0]), start + batch_size)
        output = {'document_tokens':[],
                    'document_lengths':[],
                    'answer_labels':[],
                    'answer_lengths': [],
                    'question_input_tokens':[],
                    'question_output_tokens':[],
                    'question_lengths':[],
                    'valid_tokens': [],
                    'valid_tokens_length': []}
        
        for index,inp in enumerate(inputs):
            #maxD = max(inputs[1][start:start+batch_size])
            maxD = max_document_len
            maxA = max(inputs[3][start:start+batch_size])
            maxQ = max_question_len
            
            if index == 0:
                output['document_tokens'].append(inp[start:end,0:maxD])
            elif index==1:
                output['document_lengths'].append(inp[start:end])
            elif index==2:
                output['answer_labels'].append(inp[start:end,0:maxD])
            elif index==3:
                output['answer_lengths'].append(inp[start:end])
            elif index==4:
                output['question_input_tokens'].append(inp[start:end, 0:maxQ])
            elif index==5:
                output['question_output_tokens'].append(inp[start:end, 0:maxQ])
            elif index==6:
                output['question_lengths'].append(inp[start:end])
            elif index==7:
                output['valid_tokens'].append(inp[start:end, 0:reduced_glove.shape[0]])
            elif index == 8:
                output['valid_tokens_length'].append(inp[start:end])
        
        p = np.argsort(np.array(output["document_lengths"]) * -1)
        output["document_tokens"] = np.array(output["document_tokens"])[:,p]
        output["document_lengths"] = np.array(output["document_lengths"])[:,p]
        output["answer_labels"] = np.array(output["answer_labels"])[:,p]
        output["answer_lengths"] = np.array(output["answer_lengths"])[:,p]
        output["question_input_tokens"] = np.array(output["question_input_tokens"])[:,p]
        output["question_output_tokens"] = np.array(output["question_output_tokens"])[:,p]
        output["question_lengths"] = np.array(output["question_lengths"])[:,p]
        output["valid_tokens"] = np.array(output["valid_tokens"]).reshape(batch_size, -1)[:,p]
        output["valid_tokens_length"] = np.array(output["valid_tokens_length"])[:,p]
        outputs.append(output)
        start = start + batch_size
            
    return outputs


In [325]:
batch_size = 8
batch_input = createBatch([document_tokens,document_lengths,answer_labels,answer_lengths,question_input_tokens,question_output_tokens,question_lengths,valid_tokens, valid_tokens_length]
                    ,batch_size)
for b in batch_input:
    for k, v in b.items():
        b[k] = v.squeeze()
number_of_batches = len(batch_input)
print("Number of batches = ", number_of_batches)

Number of batches =  4


(8, 32)

In [354]:
def randomShuffleBatch(batch):
    random.sample(range(1, 100), 3)
    p = random.sample(range(0 ,batch['document_tokens'].shape[0]),batch['document_tokens'].shape[0]) 
    batch['document_tokens'] = batch['document_tokens'][p]
    batch['document_lengths'] = batch['document_lengths'][p]
    batch['answer_labels'] = batch['answer_labels'][p]
    batch['answer_lengths'] = batch['answer_lengths'][p]
    batch['question_input_tokens'] = batch['question_input_tokens'][p]
    batch['question_output_tokens'] = batch['question_output_tokens'][p]
    batch['question_lengths'] = batch['question_lengths'][p]
    batch['valid_tokens'] = batch['valid_tokens'][p]
    batch['valid_tokens_length'] = batch['valid_tokens_length'][p]
    

In [355]:
randomShuffleBatch(batch_input[0])

In [356]:
batch_input[0]['document_lengths']

array([22, 32, 26, 22, 21, 23, 21, 21], dtype=int32)

In [357]:
len(batch_input)

4

In [358]:
batch_input[0]['document_lengths']

array([22, 32, 26, 22, 21, 23, 21, 21], dtype=int32)

In [359]:
look_up_token_reduced(2)

'<END>'

## Model

In [360]:
USE_CUDA = torch.cuda.is_available()
#USE_CUDA = False
print('Use_CUDA={}'.format(USE_CUDA))
if USE_CUDA:
    # You can change device by `torch.cuda.set_device(device_id)`
    print('current_device={}'.format(torch.cuda.current_device()))

Use_CUDA=True
current_device=0


In [361]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [362]:
def batchwise_sample(gen, num_samples, batch_size, comp, documentEncoder, embedder):
    """
    Sample num_samples samples batch_size samples at a time from gen.
    Does not require gpu since gen.sample() takes care of that.
    """

    samples = []
    start = 0
    for i in range(int(math.ceil(num_samples/float(batch_size)))):
        batch_comp = comp[start:min(start+batch_size, num_samples)]
        samples.append(gen.sample(batch_comp.shape[0], batch_comp, documentEncoder, embedder).transpose(1,0))
        start+=batch_size

    return torch.cat(samples, START_TOKEN)[:num_samples].transpose(0,1)

In [363]:
def prepare_generator_batch(samples, start_letter=START_TOKEN, gpu=False):
    """
    Takes samples (a batch) and returns
    Inputs: samples, start_letter, cuda
        - samples: batch_size x seq_len (Tensor with a sample in each row)
    Returns: inp, target
        - inp: batch_size x seq_len (same as target, but with start_letter prepended)
        - target: batch_size x seq_len (Variable same as samples)
    """

    batch_size, seq_len = samples.size()

    inp = torch.zeros(batch_size, seq_len)
    target = samples
    inp[:, 0] = start_letter
    inp[:, 1:] = target[:, :seq_len-1]

    inp = Variable(inp).type(torch.LongTensor)
    target = Variable(target).type(torch.LongTensor)

    if gpu:
        inp = inp.cuda()
        target = target.cuda()

    return inp, target

In [364]:
torch.__version__

'0.3.1.post2'

### Document Encoder

In [365]:
class Embedder(nn.Module):
    def __init__(self, input_size, output_size):
        super(Embedder, self).__init__()
        self.embedding = nn.Embedding(input_size, output_size)
        
        self.embedding.weight = nn.Parameter(torch.from_numpy(reduced_glove).float())
        self.embedding.weight.requires_grad = False
    def forward(self, x):
        return self.embedding(x)

In [366]:
class DocumentEncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(DocumentEncoderRNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers=num_layers, batch_first= True, bidirectional=True)

    def forward(self, x, lens, hidden):
        #x = nn.utils.rnn.pack_padded_sequence(input = x, lengths = lens , batch_first=True)
        
        _, hidden = self.gru(x, hidden)
        hidden = hidden.view(1, hidden.shape[1], hidden.shape[2]*2*self.num_layers)
        return hidden
    
    def initHidden(self, num_samples):
        result = Variable(torch.zeros(2 * self.num_layers, num_samples, self.hidden_size)) #2 for BiDirectional
        if USE_CUDA:
            result = result.cuda()
        return result

### Generator

In [367]:
class Generator(nn.Module):

    def __init__(self, embedding_dim, embeddings,  hidden_dim, vocab_size, max_seq_len, gpu=False):
        
        super(Generator, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.embeddings = embeddings
        self.max_seq_len = max_seq_len
        self.vocab_size = vocab_size
        self.gpu = gpu

        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first= False)
        self.gru2out = nn.Linear(hidden_dim, vocab_size)

    def init_hidden(self, comp, comp_lens, num_examples, documentEncoder, embedder):
        comp = embedder(comp).float()
        h = documentEncoder.forward(comp, comp_lens, documentEncoder.initHidden(num_examples))
        #h = Variable(torch.zeros(1, batch_size, self.hidden_dim))
        if self.gpu:
            h = h.cuda()
        
        return h

    def forward(self, inp, hidden, doc_enc_hidden): #Make sure hidden comes from the DocumentEncoder...
        """
        Embeds input and applies GRU one token at a time (seq_len = 1)
        """
        
        emb = self.embeddings(inp)                              # batch_size x embedding_dim
        emb = emb.view(1, -1, self.embedding_dim).float()       # 1 x batch_size x embedding_dim
        out, hidden = self.gru(emb, hidden) # 1 x batch_size x hidden_dim (out)
        #out = torch.cat((out, doc_enc_hidden),dim=-1)
        out = out.view(-1, self.hidden_dim)
        out = self.gru2out(out)  # batch_size x vocab_size
        out = F.log_softmax(out, dim=-1)
        return out, hidden
    
    def batchNLLLoss(self, inp, target, comp, comp_lens, documentEncoder, embedder, num_samples = batch_size):
        """
        Returns the NLL Loss for predicting target sequence.
        Inputs: inp, target
            - inp: batch_size x seq_len
            - target: batch_size x seq_len
            inp should be target with <s> (start letter) prepended
        """

        loss_fn = nn.NLLLoss()
        batch_size, seq_len = inp.size()
        inp = inp.permute(1, 0)           # seq_len x batch_size
        target = target.permute(1, 0)     # seq_len x batch_size
        doc_enc_hidden = self.init_hidden(comp, comp_lens, num_samples, documentEncoder, embedder)
        h = doc_enc_hidden

        loss = 0
        for i in range(seq_len):
            out, h = self.forward(inp[i], h, doc_enc_hidden)
            loss += loss_fn(out, target[i])

        return loss     # per batch

    def sample(self, num_samples, comp, comp_lens,  documentEncoder, embedder, start_letter=START_TOKEN):
        """
        Samples the network and returns num_samples samples of length max_seq_len.
        Outputs: samples, hidden
            - samples: num_samples x max_seq_length (a sampled sequence in each row)
        """

        samples = torch.zeros(num_samples, self.max_seq_len).type(torch.LongTensor)

        doc_enc_hidden = self.init_hidden(comp, comp_lens, num_samples, documentEncoder, embedder)
        inp = torch.autograd.Variable(torch.LongTensor([start_letter]*num_samples))

        if self.gpu:
            samples = samples.cuda()
            inp = inp.cuda()
        h = doc_enc_hidden
        for i in range(self.max_seq_len):
            out, h = self.forward(inp, h, doc_enc_hidden)               # out: num_samples x vocab_size
            _, out = torch.max(torch.exp(out), dim=-1)  # num_samples x 1 (sampling from each row)
            # out = torch.multinomial(torch.exp(out), 1)  # num_samples x 1 (sampling from each row)
            samples[:, i] = out.data

            inp = out.view(-1)

        return samples
    
    def randomSample(self, num_samples, comp, documentEncoder, embedder, start_letter=START_TOKEN, num_rand = 5, minProb = 0.3):
        
        samples = torch.zeros(num_samples, self.max_seq_len).type(torch.LongTensor)

        doc_enc_hidden = self.init_hidden(comp, num_samples, documentEncoder, embedder)
        inp = torch.autograd.Variable(torch.LongTensor([start_letter]*num_samples))

        if self.gpu:
            samples = samples.cuda()
            inp = inp.cuda()
        h = doc_enc_hidden
        for i in range(self.max_seq_len):
            out, h = self.forward(inp, h, doc_enc_hidden)               # out: num_samples x vocab_size
            out_sample = torch.multinomial(torch.exp(out), num_rand)  # num_samples x 1 (sampling from each row)
            out = out_sample[random.randint(0,num_rand)]
            samples[:, i] = out.data

            inp = out.view(-1)

        return samples

    

    def batchSuppressionLoss(self, inp,valid_toks,valid_tok_len, comp,documentEncoder, embedder, num_samples = batch_size):
        batch_size, seq_len = inp.size()
        inp = inp.permute(1, 0)           # seq_len x batch_size
        doc_enc_hidden = self.init_hidden(comp, num_samples, documentEncoder, embedder)
        h = doc_enc_hidden
        
        loss = 0
        for i in range(seq_len):
            out, h = self.forward(inp[i], h, doc_enc_hidden)
            _, vocab_size_o = out.size()
            #out = torch.exp(out)
            #l1 = torch.sum(torch.mul(out, (1-valid_toks)).float(), dim = 1)
            l2 = torch.sum(torch.mul(out, valid_toks.float()), dim = 1)
            
            #l1 = torch.div(l1, (vocab_size_o - valid_tok_len))
            #loss += (torch.sum(l1) - torch.sum(l2))
            loss += -l2
        return loss
    
    def batchDegeneracyLoss(self, inp, comp,comp_lens,documentEncoder, embedder, num_samples = batch_size):
        batch_size, seq_len = inp.size()
        inp = inp.permute(1, 0)           # seq_len x batch_size
        doc_enc_hidden = self.init_hidden(comp, comp_lens,  num_samples, documentEncoder, embedder)
        h = doc_enc_hidden
        loss = 0
        for i in range(seq_len):
            out, h = self.forward(inp[i], h, doc_enc_hidden)
            out = torch.exp(out)
            out = out.unsqueeze(2)
            a1 = out.transpose(1,2)
            a2 = torch.log(out)
            loss += torch.bmm(a1, a2)
        loss = torch.sum(loss) / (batch_size * seq_len)
        return loss
        

    def batchPGLoss(self, inp, target, reward, comp, documentEncoder, embedder):
        """
        Returns a pseudo-loss that gives corresponding policy gradients (on calling .backward()).
        Inspired by the example in http://karpathy.github.io/2016/05/31/rl/
        Inputs: inp, target
            - inp: batch_size x seq_len
            - target: batch_size x seq_len
            - reward: batch_size (discriminator reward for each sentence, applied to each token of the corresponding
                      sentence)
            inp should be target with <s> (start letter) prepended
        """

        batch_size, seq_len = inp.size()
        inp = inp.permute(1, 0)          # seq_len x batch_size
        target = target.permute(1, 0)    # seq_len x batch_size
        h = self.init_hidden(comp, batch_size, documentEncoder, embedder)

        loss = 0
        for i in range(seq_len):
            out, h = self.forward(inp[i], h)
            # TODO: should h be detached from graph (.detach())?
            for j in range(batch_size):
                print(target.data[i][j], out[j][target.data[i][j]])
                loss += -out[j][target.data[i][j]]*reward[j]     # log(P(y_t|Y_1:Y_{t-1})) * Q

        return loss/batch_size

In [368]:
embedder = Embedder(input_size = reduced_glove.shape[0], output_size = reduced_glove.shape[1])

In [369]:
hidden_size = 50

documentEncoder = DocumentEncoderRNN(reduced_glove.shape[1], hidden_size)

gen = Generator(embedding_dim = reduced_glove.shape[1], 
                embeddings = embedder, 
                hidden_dim = hidden_size * 2, 
                vocab_size = reduced_glove.shape[0], 
                max_seq_len = max_question_len, 
                gpu=USE_CUDA)

if USE_CUDA:
    gen= gen.cuda()
    documentEncoder = documentEncoder.cuda()


In [370]:
def train_generator_MLE(gen, gen_opt, batch_input, epochs):
    """
    Max Likelihood Pretraining for the generator
    """
    for epoch in range(epochs):
        print('epoch %d : ' % (epoch + 1), end='')
        sys.stdout.flush()
        total_loss = 0
        total_loss_2 = 0
        total_loss_3 = 0
        print(" ")
        for i in range(len(batch_input)):
            print('batch %d/%d : ' % (i + 1, len(batch_input)), end='\r')
            randomShuffleBatch(batch_input[i])
            inp = batch_input[i]['question_input_tokens']
            target = batch_input[i]['question_output_tokens'] 
            comp =  batch_input[i]['document_tokens']
            comp_lens = batch_input[i]['document_lengths']
            valid_toks = batch_input[i]['valid_tokens']
            valid_tok_len = batch_input[i]['valid_tokens_length']
            if USE_CUDA:
                inp = torch.from_numpy(inp).long().cuda()
                target = Variable(torch.from_numpy(target).long().cuda())
                comp = Variable(torch.from_numpy(comp).long().cuda())
                valid_toks = Variable(torch.from_numpy(valid_toks).float().cuda())
                valid_tok_len = Variable(torch.from_numpy(valid_tok_len).float().cuda())
                #comp_lens = Variable(torch.from_numpy(comp_lens).float().cuda())
            else:
                inp = torch.from_numpy(inp).long()
                target = Variable(torch.from_numpy(target).long())
                comp = Variable(torch.from_numpy(comp).long())
                valid_toks = Variable(torch.from_numpy(valid_toks).float())
                valid_tok_len = Variable(torch.from_numpy(valid_tok_len).float())
                #comp_lens = Variable(torch.from_numpy(comp_lens).float())
                
            gen_opt.zero_grad()
            
            #loss3 = gen.batchSuppressionLoss(inp,valid_toks,valid_tok_len,comp,documentEncoder, embedder, inp.shape[0])

            #loss2 = gen.batchDegeneracyLoss(inp, comp, documentEncoder, embedder, inp.shape[0])
            
            loss = gen.batchNLLLoss(inp,target,comp,comp_lens.tolist(),documentEncoder, embedder, inp.shape[0])
            
            net_loss = loss
            net_loss.backward()
            gen_opt.step()

            total_loss += loss.data[0]
#            total_loss_2+= loss2.data[0]
#            total_loss_3 += loss3.data[0] 
            sys.stdout.flush()


        print(' Total Loss = %.4f, Degeneracy Loss = %.4f, Suppression Loss = %.4f' % 
              (total_loss, 0, 0))

In [371]:
train_param_gen = []
train_param_disc = []

for model in [gen, documentEncoder]:
    train_param_gen += [p for p in model.parameters() if p.requires_grad]
    
gen_optimizer = optim.Adam(train_param_gen, lr=1e-2)




In [372]:
print("Training Generator")
train_generator_MLE(gen,gen_optimizer, batch_input, 20000)


Training Generator
epoch 1 :  
 Total Loss = 823.7488, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 2 :  
 Total Loss = 402.5255, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 3 :  
 Total Loss = 263.8961, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 4 :  
 Total Loss = 231.1911, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 5 :  
 Total Loss = 210.7619, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 6 :  
 Total Loss = 197.8275, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 7 :  
 Total Loss = 186.3703, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 8 :  
 Total Loss = 174.9072, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 9 :  
 Total Loss = 163.2576, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 10 :  
 Total Loss = 153.0750, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 11 :  
 Total Loss = 139.8184, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 12 : 

epoch 94 :  
 Total Loss = 12.0974, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 95 :  
 Total Loss = 11.8142, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 96 :  
 Total Loss = 11.6309, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 97 :  
 Total Loss = 11.5712, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 98 :  
 Total Loss = 11.2103, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 99 :  
 Total Loss = 11.8317, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 100 :  
 Total Loss = 11.1205, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 101 :  
 Total Loss = 11.7454, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 102 :  
 Total Loss = 11.4791, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 103 :  
 Total Loss = 12.1157, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 104 :  
 Total Loss = 11.6992, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 105 :  
 Total Loss =

epoch 187 :  
 Total Loss = 9.3317, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 188 :  
 Total Loss = 9.4532, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 189 :  
 Total Loss = 8.9493, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 190 :  
 Total Loss = 9.1360, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 191 :  
 Total Loss = 9.7589, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 192 :  
 Total Loss = 8.9335, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 193 :  
 Total Loss = 9.2254, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 194 :  
 Total Loss = 9.0575, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 195 :  
 Total Loss = 9.4537, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 196 :  
 Total Loss = 9.9477, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 197 :  
 Total Loss = 9.5717, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 198 :  
 Total Loss = 9.67

epoch 280 :  
 Total Loss = 8.9666, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 281 :  
 Total Loss = 8.6024, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 282 :  
 Total Loss = 8.9784, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 283 :  
 Total Loss = 9.5848, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 284 :  
 Total Loss = 8.9781, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 285 :  
 Total Loss = 9.0212, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 286 :  
 Total Loss = 9.1980, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 287 :  
 Total Loss = 9.7980, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 288 :  
 Total Loss = 8.9693, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 289 :  
 Total Loss = 8.9767, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 290 :  
 Total Loss = 8.9665, Degeneracy Loss = 0.0000, Suppression Loss = 0.0000
epoch 291 :  
 Total Loss = 9.52

KeyboardInterrupt: 

In [375]:
batch_num = 2
num_examples = 5
start = 0
end = start + num_examples
x = gen.sample(
    num_examples, 
    torch.from_numpy(batch_input[batch_num]['document_tokens'][start:end]).cuda(),
    batch_input[batch_num]['document_lengths'][start:end],
    documentEncoder, 
    embedder)
#loss2 = gen.batchSuppressionLoss(inp,valid_toks,comp,documentEncoder, embedder, inp.shape[0]) 
#loss = gen.batchNLLLoss(inp,target,comp,documentEncoder, embedder, inp.shape[0])
for i in range(x.shape[0]):
    print("----------------------------------------------------------------------------------------------------")
    print("Comprehension : ")
    printTokens(batch_input[batch_num]['document_tokens'][i])
    print("*****************************************************************************************************")
    print("Ground Truth Question : ")
    printTokens(batch_input[batch_num]['question_input_tokens'][i])
    print("*****************************************************************************************************")
    print("Generated Question : ")
    printTokens(x[i])
    print("----------------------------------------------------------------------------------------------------")


----------------------------------------------------------------------------------------------------
Comprehension : 
for more than 1,500 years . important contributions from the medieval muslim world include ibn wahshiyya 's nabatean agriculture , abū ḥanīfa dīnawarī <END> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>  
*****************************************************************************************************
Ground Truth Question : 
<START> besides the greeks , what other culture contributed to the study of botany ? <PAD> <PAD> <PAD> <PAD> <PAD>  
*****************************************************************************************************
Generated Question : 
what does a lush environment allow hunter-gatherers to be ? <END> ? <END> ? <END> ? <END> ? <END> ?  
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Com

In [248]:
reversed(np.array([1,2,3,4,5]))

In [ ]:
inp_test = inp_l.permute(1, 0)           # seq_len x batch_size
h = gen.init_hidden(comp_l, inp_l.shape[0], documentEncoder, embedder)

In [ ]:
loss = 0
seq_len = max_question_len
for i in range(2,3):
    out, h = gen.forward(inp_test[i], h)
    x = 0
    for j in torch.multinomial(torch.exp(out), 5) : 
        for k in j:
            print(torch.exp(out[x][k]).data[0],look_up_token_reduced(k.data[0]), sep = ' ', end = ' ')
        print(" ")
        x+=1

In [ ]:
batch_num = 0
num_examples_l = batch_input[batch_num]['document_tokens'].shape[0],  #Number of examples
comp_l = Variable(torch.from_numpy(batch_input[batch_num]['document_tokens']).long()).cuda() #Document to torch Variable
inp_l = Variable(torch.from_numpy(batch_input[batch_num]['question_input_tokens']).long()).cuda()
target_l = Variable(torch.from_numpy(batch_input[batch_num]['question_output_tokens']).long()).cuda()
valid_toks_l = Variable(torch.from_numpy(batch_input[batch_num]['valid_tokens']).float()).cuda()
valid_toks_length_l = Variable(torch.from_numpy(batch_input[batch_num]['valid_tokens_length']).float()).cuda()
#documentEncoder, 
#embedder

#supLoss = gen.batchSuppressionLoss(inp_l,valid_toks_l,valid_toks_length_l,comp_l,documentEncoder, embedder, inp_l.shape[0])
#degenLoss = gen.batchDegeneracyLoss(inp_l, comp_l, documentEncoder, embedder, inp_l.shape[0])
nllLoss = gen.batchNLLLoss(inp_l,target_l,comp_l,documentEncoder, embedder, inp_l.shape[0])

print(supLoss)
print(degenLoss)
print(nllLoss)


In [ ]:
def train_generator_PG(gen, gen_opt, dis, documentEncoder, embedder):
    """
    The generator is trained using policy gradients, using the reward from the discriminator.
    Training is done for num_batches batches.
    """
    
    total_loss = 0
    for batch in batch_input:
        comp = Variable(torch.from_numpy(batch['document_tokens']))
        if USE_CUDA:
            comp = comp.cuda()
            
        s = gen.sample(batch['document_tokens'].shape[0], comp, documentEncoder, embedder)
        inp, target = prepare_generator_batch(s, start_letter=START_TOKEN, gpu=USE_CUDA)
        rewards = disc.batchClassify(target, target.shape[0], comp, documentEncoderDisc, embedder)

        gen_opt.zero_grad()
        pg_loss = gen.batchPGLoss(inp, target, rewards, comp, documentEncoder, embedder)
        total_loss += pg_loss.data[0]
        pg_loss.backward()
        gen_opt.step()
    

    print('Total PG Loss : ',  total_loss)